In [1]:
from league_data_dict import league_data_dict
from bs4 import BeautifulSoup

import requests, os, datetime
import pandas as pd
import numpy as np

Note
- File hierarchy: data -> tables -> file

In [2]:
# Change these -------------
end_year = 2022 # one after last year you want
chosen_league = 'English Premier League'
# --------------------------

start_year = league_data_dict[chosen_league]['First Year - Table']
league = league_data_dict[chosen_league]['Code'] # put league or competition here
folder_name = league_data_dict[chosen_league]['Folder']


file_name = '{}_{}_tables.csv'.format(start_year, end_year-1)
url = "https://www.espn.com/soccer/standings/_/league/{}/season/{}"

# GP W D L GF GA GD P
cols = ['Place', 'Team', 'GP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'P'] # may need to change if other leagues have other cols
table_df = pd.DataFrame([], columns=cols)

for year in range(start_year, end_year): # possibly change these
    r = requests.get(url.format(league, year))
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    teams_soup = soup.find_all('table')[0].find_all('div', class_='team-link flex items-center clr-gray-03')
    
    teams = []
    for team in teams_soup:
        teams.append([team.contents[0].text, team.find('img').get('title')])

    stats = soup.find_all('span', class_='stat-cell')

    # convert to np array and reshape
    stats = [item.text for item in stats]
    np_stats = np.array(stats).reshape(-1, 8)

    final_arr = np.concatenate((np.array(teams).reshape(-1, 2), np_stats), axis=1)
    temp_df = pd.DataFrame(final_arr, columns=cols)
    temp_df['Year'] = year
    table_df = table_df.append(temp_df, ignore_index=True)

In [3]:
# if folder doesn't exist, make it
tpath = os.path.join('data', folder_name, 'tables')
if not os.path.exists(tpath):
    os.makedirs(tpath)

table_df['Year'] = table_df['Year'].astype(int)
table_df.to_csv(os.path.join('data', folder_name, 'tables', file_name), index=False)
table_df.to_csv(os.path.join('data', folder_name, 'tables', 'all_tables.csv'), index=False)

In [4]:
table_df.tail(20)

,Place,Team,GP,W,D,L,GF,GA,GD,P,Year
400,1,CHE,5,4,1,0,12,1,+11,13,2021
401,2,LIV,5,4,1,0,12,1,+11,13,2021
402,3,MAN,5,4,1,0,13,4,+9,13,2021
403,4,BHA,5,4,0,1,7,4,+3,12,2021
404,5,MNC,5,3,1,1,11,1,+10,10,2021
405,6,EVE,5,3,1,1,10,7,+3,10,2021
406,7,TOT,5,3,0,2,3,6,-3,9,2021
407,8,WHU,5,2,2,1,11,7,+4,8,2021
408,9,BRN,5,2,2,1,5,2,+3,8,2021
409,10,AVL,5,2,1,2,8,7,+1,7,2021
